In [2]:
'''
Naive Bayes 이론에 근거한 통계적 분류기
 - GaussianNB : x변수가 실수이고, 정규분포 데이터에 적용   
 - BeBrnoulliNB : 베르누이 분포(이항분포) 데이터에 적용(y : 0 or 1)
 - MultinomialNB : 고차원의 텍스트 분류(tf-idf)에 적용(Sparse matrix) 
'''

import pandas as pd
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from scipy import stats # 확률분포 검정 

#########################
## y변수 다항분류
#########################
data = pd.read_csv('C:/Users/hyebin/Desktop/Python_ML/data/iris.csv')
data.head() # x변수 실수형 


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
cols = list(data.columns)
x_cols = cols[:4]
y_col = cols[-1]

# x의 정규성 검정 
test_stat, pvalue = stats.shapiro(data[x_cols[1]]) # sample data 
print("test_stat = %.5f, pvalue = %.5f"%(test_stat, pvalue))

test_stat = 0.98492, pvalue = 0.10113


In [5]:
# 1. train/test data set 구성 
train_set, test_set = model_selection.train_test_split(
     data, test_size=0.3, random_state=123) # seed값 

print(train_set.shape) # (90, 5)
print(test_set.shape) # (60, 5)

(105, 5)
(45, 5)


In [6]:
# 2. NB model 생성 
gnb = GaussianNB()
model = gnb.fit(train_set[x_cols], train_set[y_col]) # y 집단변수 

In [7]:
# 3. model 예측치 
preds = model.predict(test_set[x_cols])

In [8]:
# 4. model 평가 

# 1) accuracy
model_acc = metrics.accuracy_score(test_set[y_col], preds)
print('accuracy =', model_acc) # accuracy = 0.9555555555555556

accuracy = 0.9555555555555556


In [11]:
# 2) confusion matrix
con_mat=pd.crosstab(preds, test_set[y_col])
con_mat

Species,setosa,versicolor,virginica
row_0,,,
setosa,18,0,0
versicolor,0,10,2
virginica,0,0,15


In [12]:
tab = con_mat.setosa[0]+con_mat.versicolor[1]+con_mat.virginica[2]
print(tab / len(test_set))

0.9555555555555556


In [13]:
# 3) f1 score : average=[None, 'micro', 'macro', 'weighted'].
f1_score = metrics.f1_score(test_set[y_col], preds, average='micro') 
print('f1 score =', f1_score)

f1 score = 0.9555555555555556


In [16]:
###############################
### news groups 분류 
###############################

from sklearn.naive_bayes import MultinomialNB # tfidf 문서분류 
from sklearn.datasets import fetch_20newsgroups # news 데이터셋 
from sklearn.feature_extraction.text import TfidfVectorizer# 희소행렬 

# 1. dataset 가져오기 
newsgroups = fetch_20newsgroups(subset='all') # train/test load 

In [18]:
print(newsgroups.target_names,"\n")
print(len(newsgroups.target_names)) # 20개 뉴스 그룹 

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc'] 

20


In [19]:
'''
 문서단어 행렬 : 희소행렬(sparse matrix)  
 텍스트 데이터로 예측모델 또는 군집 모델을 생성하기 텍스트를 적절한 숫자 값의 벡터 생성 
'''

# 2. train set 선택 : 4개 뉴스 그룹  
#cats = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space']
cats = list(newsgroups.target_names)[:4]

news_train = fetch_20newsgroups(subset='train',categories=cats)
news_train.data # texts
news_train.target # 0 ~ 3

array([3, 2, 2, ..., 0, 1, 1], dtype=int64)

In [20]:
# 3. sparse matrix
vectorizer = TfidfVectorizer()
sparse_train = vectorizer.fit_transform(news_train.data)
sparse_train.shape # (2245, 62227)

(2245, 62227)

In [22]:
# 4. NB 모델 생성 
model = MultinomialNB() # alpha=.01 (default=1.0)

'''
스무딩(smoothing)
표본 데이터의 수가 적은 경우에는 베르누이 모수가 0 또는 1이라는 극단적인 
모수 추정값이 나오는 것을 방지하기 위해서 0과 1의 가상 표본 데이터를 
추가하여 극단적인 추정값이 0.5에 가깝게 만드는 기능의미 한다.
alpha 속성은 스무딩의 정도를 지정하는 가중치 역할 
'''

#help(MultinomialNB)
model.fit(sparse_train, news_train.target) # 훈련셋 적용 
'''
주의 :
vectorizer.fit_transform(news_train.data)
vectorizer.transform(news_test.data)
'''

MultinomialNB()

In [23]:
# 5. test dataset 4개 뉴스그룹 대상 : 희소행렬
news_test = fetch_20newsgroups(subset='test', categories=cats)
sparse_test = vectorizer.transform(news_test.data)
sparse_test.shape # (1494, 62227)

(1494, 62227)

In [25]:
# 6. model 예측치 
target_pred = model.predict(sparse_test) # 예측치 

# 7. model 평가 : average=[None, 'micro', 'macro', 'weighted'].
metrics.f1_score(news_test.target, target_pred, average='micro') 

0.8520749665327979

In [27]:
# real value vs predict
print(news_test.target[:20],"\n")
print(target_pred[:20])

[1 1 1 2 3 1 3 1 0 3 3 3 0 2 3 1 0 3 2 2] 

[1 2 1 2 3 1 3 1 0 3 1 3 0 2 3 1 0 3 0 2]
